# Introduction

This Jupyter notebook, seeks to replicate the results from the paper, "Online Parameter Estimation Methods for Adaptive Cruise Control" by et al. Wang. This notebook will  seek to comprehend the computational robustness of RLS within Adaptive Cruise Control dynamical systems.                                                         

# Mathematical Model

We adopt the **Constant Time Headway–Relative Velocity (CTH–RV)** car–following model:

$$
\dot v(t)
= \alpha\bigl(s(t) - \tau\,v(t)\bigr)
  + \beta\,\bigl(u(t) - v(t)\bigr)
\quad\text{(continuous‐time)}\tag{1}
$$

where  
- $s(t)$ is the space gap,  
- $v(t)$ is the ego-vehicle speed,  
- $\Delta v(t)=u(t)-v(t)$ is the relative speed to the leader,  
- $\theta=[\alpha,\beta,\tau]^\top$ are the model parameters.

Discretizing with a forward-Euler step $\Delta T$ yields:

$$
v_{k+1}
= v_k
  + \alpha\,(s_k - \tau\,v_k)\,\Delta T
  + \beta\,(u_k - v_k)\,\Delta T
\tag{5}
$$

Rewriting in linear-regression form:

$$
v_{k+1}
= \gamma_1\,v_k + \gamma_2\,s_k + \gamma_3\,u_k,
\quad
\begin{aligned}
\gamma_1 &= 1 - (\alpha\,\tau + \beta)\,\Delta T,\\
\gamma_2 &= \alpha\,\Delta T,\\
\gamma_3 &= \beta\,\Delta T.
\end{aligned}
\tag{6}
$$

Stacking data for $k=1,\dots,K-1$:

$$
Y = X\,\gamma,
\quad
Y = 
\begin{bmatrix}
v_2\\
\vdots\\
v_K
\end{bmatrix},
\quad
X =
\begin{bmatrix}
v_1 & s_1 & u_1\\
\vdots & \vdots & \vdots\\
v_{K-1} & s_{K-1} & u_{K-1}
\end{bmatrix}
\tag{8}
$$

The **RLS** algorithm then updates $\gamma$ sequentially as each $X_k,Y_k$ arrives:

$$
\begin{aligned}
K_k &= \frac{P_{k-1}\,X_k}{1 + X_k^\top P_{k-1} X_k},\\
\gamma_k &= \gamma_{k-1} + K_k\bigl(Y_k - X_k^\top\gamma_{k-1}\bigr),\\
P_k &= \bigl(I - K_k\,X_k^\top\bigr)\,P_{k-1},
\end{aligned}
\tag{9}
$$

with initial guess $\gamma_{0}$ and covariance $\boldsymbol{P}_0$.


# Algorithm & Scenarios: 

In [1]:
"""
Recursive Least Squares (RLS) Implementation for ACC Parameter Estimation:

This script implements the Recursive Least Squares algorithm to estimate parameters 
of an Adaptive Cruise Control (ACC) model. The ACC model is a car-following model
that describes how a following vehicle adjusts its velocity based on the lead vehicle
and the space gap between them.

The model uses three parameters:
    - alpha: Sensitivity to space gap
    - beta: Sensitivity to velocity difference
    - tau: Desired time headway

The script performs the following steps:
    1. Generate synthetic data using known parameters
    2. Implement RLS to estimate parameters from simulated data
    3. Compare estimated parameters with true values
    4. Generate plots to visualize convergence and model behavior
"""

# Needed Dependecies: 
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import os

# RLS Function(s): 
def invert_gamma(gamma, dt):
    """
    Given gamma1, gamma2, gamma3 = alpha, beta, tau, we are able to directly
    solve for alpha, beta, tau by using the following equations algebraically:
    - alpha = gamma2 / dt
    - beta = gamma3 / dt
    - tau = ((1 - gamma1 - gamma3) / gamma2)
    """
    gamma1, gamma2, gamma3 = gamma

    alpha = gamma2/dt
    beta  = gamma3/dt

    if abs(alpha) < 1e-8: # Added tolerance for numerical stability
        tau = 0.0
    else:
        tau = ((1 - gamma1 - gamma3) / gamma2)
    return alpha, beta, tau


time = 900  # Total number of time steps
dt = 0.1 # Time step in seconds

# Seed for Reproducibility:
np.random.seed(0)

# True theta vector: 
true_theta = np.array([0.08, 0.12, 1.5]) # true_theta[0] = alpha, true_theta[1] = beta, true_theta[2] = tau

In [2]:
"""
Initial Conditions & Restraints: 
"""
s0 = 60.0 # SI: m
v0 = 33.0 # SI: m/s (approximately 119 km/h)
u0 = 31.0 # SI: m/s (approximately 112 km/h)

# Maximum acceleration/deceleration constraint (m/s²)
dv_max = 3.0  

# Lead Trajectory Generation: 

This section will generate the velocity profiles for the lead vehicle $u(t)$.

There will be a total of 4 profiles that will be available to work with. 

- 1. Curve Simulation: 
    - Simulates a vehicle navigating a road curve
    - Sharp deceleration entering the curve (300-400 time steps)
    - Acceleration exiting the curve (340-360 time steps)
    - Random small variations anywhere else

- 2. Random Walk (Default Simulation):
    - Adds small random increments to velocity at each time step
    - Crates more unpredictable but still realistic velocity changes
    - Uses normal distribution with mean, $\mu=0$, $\sigma=0.2$ m/s

- 3. Suburban Simulation: 
    - Simulates several hard stops in suburban areas
    - Simulation tests robustness of RLS with a high amount of stops
    - Will create more unpredictable but still realistic velocity changes
    - 

In [ ]:
# Key to decide which scenario to use:
# Change this key to switch between scenarios
# 1 = Random Walk Model
# 2 = Curve Simulation 
# 3 = Suburban Driving Simulation
# 4 - Aggresive Driving Simulation
scenario_key = 1

if scenario_key == 1:
    for i in range(1, 900):
        u_t[i] = u_t[i-1] + np.random.normal(loc=0, scale=0.2) # random increments
elif scenario_key == 2:
    # Simulated Curve:
    for i in range(1, 900):
        if 300 <= i < 340:
            u_t[i] = u_t[i-1] - 2.0  # sudden deceleration entering curve
        elif 340 <= i < 360:
            u_t[i] = u_t[i-1] + 1.0  # acceleration exiting curve
        else:
            u_t[i] = u_t[i-1] + np.random.normal(0, 0.2)
        # Safety: keep velocity in physical range
        u_t[i] = np.clip(u_t[i], 0, 35)
elif scenario_key == 3:
    for i in range(1, time):
        if i % 100 < 20:
            u_t[i] = max(u_t[i - 1] - 3.0, 0)  # Stop zone
        else:
            u_t[i] = min(u_t[i - 1] + 0.5, 30) + np.random.normal(0, 0.2)

elif scenario_key == 4: # Scenario 4 implicitly
    for i in range(1, time):
        u_t[i] = u_t[i - 1] + np.random.normal(0, 1.0)  # Aggressive, erratic driving
        u_t[i] = np.clip(u_t[i], 0, 40)

else:
    raise ValueError("Invalid scenario_key. Must be 1-4.")